# Final Training
This notebook contains the final training that is used in the analysis.

Set the name of this training, and the job number it is based on. This will keep the resulting output xml files seperate below.

In [1]:
jobId = 106
training_name = "full_event_grad1000"

## Initalization

In [2]:
from bdt_training_scikit_tools import load_trimmed_sample, default_training_variable_list, \
    test_train_samples, prep_samples, default_training, calc_performance, get_fraction_of_events
import matplotlib.pyplot as plt
plt.rc('font', size=14)
from matplotlib.colors import LogNorm
import pandas as pd
import numpy as np
import multiprocessing as mp
import itertools

In [3]:
training_file_stub = 'training_{0}_{1}'.format(jobId, training_name)

## Load data

In [4]:
%%time
input_events = load_trimmed_sample(jobId)

Job 106:
  BIB: 800000 events
  Multijet: 800000 events
  Signal: 800000 events
  [800000, 800000, 504190]
Wall time: 29.9 s


In [5]:
events_25 = get_fraction_of_events(input_events, 0.25)
print ([len(i.index) for i in events_25])

[201018, 200123, 126005]


In [6]:
events_to_use = input_events

## Training
Default training variables. Likely were arrived at by analysis in the Training Variables workbook.

In [7]:
training_variables = ['EnergyDensity',
 'BIBDeltaTimingM',
 'JetPt',
 'HadronicLayer1Fraction',
 'ShowerCenter',
 'JetLat',
 'FirstClusterRadius',
 'JetLong',
 'MaxTrackPt',
 'PredictedLxy',
 'BIBDeltaTimingP',
 'PredictedLz',
 'SumPtOfAllTracks']

Run the training to get a bdt back.

In [8]:
%%time
# Split into testing and training samples
train, test = test_train_samples(events_to_use)

# Prep samples for training
all_events, all_events_class, training_weight, evaluation_weight = prep_samples(train[0], train[1], train[2], training_variable_list=training_variables)

Wall time: 3.2 s


In [9]:
%%time
bdt = default_training(all_events, training_weight, all_events_class)

Wall time: 6h 53min 4s


Calculate the performance for this training

In [10]:
%%time
calc_performance(bdt, input_events, training_variables=training_variables)

Wall time: 2min 50s


{'BIBBack': 617119616.3334253,
 'BIBEff': 0.6472625,
 'BIBSsqrtB': 20.844225245284438,
 'BIBTotalCount': 800000,
 'BIBTotalWeight': 800000.0,
 'BIBinBIB': 517810.0,
 'BIBinHSS': 18788.0,
 'BIBinMJ': 263402.0,
 'HSSBack': 29770962.38499081,
 'HSSEff': 0.9861163450286599,
 'HSSSsqrtB': 91.12256750078656,
 'HSSTotalCount': 504190,
 'HSSTotalWeight': 504190.0,
 'HSSinBIB': 3521.0,
 'HSSinHSS': 497190.0,
 'HSSinMJ': 3479.0,
 'MJBack': 266881.0,
 'MJEff': 0.7403696296329428,
 'MJSsqrtB': 3570673.3571174936,
 'MJTotalCount': 800000,
 'MJTotalWeight': 2491496926.202845,
 'MJinBIB': 617116095.3334253,
 'MJinHSS': 29752174.38499081,
 'MJinMJ': 1844628656.484416}

In [11]:
from sklearn.externals import joblib
joblib.dump(bdt, training_file_stub + '.pkl') 

['training_106_full_event_grad1000.pkl']

## Conversion to TMVA format

In [12]:
import mlglue
import mlglue.tree
from sklearn.externals import joblib
bdt1 = joblib.load(training_file_stub + '.pkl')
bdtGeneral = mlglue.tree.BDTsklearn(bdt1, list(all_events.columns), ['BIB', 'MJ', 'Signal'])
bdtGeneral.to_tmva(training_file_stub + ".xml")

## Performance
Look at some generic performance plots for the bdt.

In [13]:
perf_events, perf_events_class, perf_training_weight, perf_evaluation_weight = prep_samples(input_events[0], input_events[1], input_events[2], training_variable_list=training_variables)

In [14]:
predicted_class = bdt1.predict(perf_events)

In [15]:
fig = plt.figure(figsize=(15,15))

ax = plt.subplot(221)
ax.hist(predicted_class[perf_events_class.Class == 0], bins=3)
ax.set_xlabel('Class')
ax.set_title('BIB')

ax = plt.subplot(222)
ax.hist(predicted_class[perf_events_class.Class == 1], bins=3)
ax.set_xlabel('Class')
ax.set_title('MJ')

ax = plt.subplot(223)
ax.hist(predicted_class[perf_events_class.Class == 2], bins=3)
ax.set_xlabel('Class')
ax.set_title('HSS')

Text(0.5,1,'HSS')

In [16]:
predicted_prob = bdt1.predict_proba(perf_events)

In [17]:
fig = plt.figure(figsize=(15,10))

ax = plt.subplot(221)
ax.hist(predicted_prob[perf_events_class.Class == 0], label=('BiB', "MJ", 'HSS'), bins=50, histtype = 'step')
ax.set_xlabel('Weight')
ax.set_title('Class: BIB')
ax.legend()

ax = plt.subplot(222)
ax.hist(predicted_prob[perf_events_class.Class == 1], label=('BiB', "MJ", 'HSS'), bins=50, histtype = 'step')
ax.set_xlabel('Weight')
ax.set_title('Class: MJ')
ax.legend()

ax = plt.subplot(223)
ax.hist(predicted_prob[perf_events_class.Class == 2], label=('BiB', "MJ", 'HSS'), bins=50, histtype = 'step')
ax.set_xlabel('Weight')
ax.set_title('Class: HSS')
ax.legend()

## TMVA Cross Check
We always have trouble with that. We need to look to see what we can expect with TMVA. Below there is Bib, MJ, and Signal, all three classes. 5 events are taken (first jet pt eta phi are shown along with event number so they can be correlated with the real full root trees). The full inputs are shown, in the proper order to be fed to the bdt. Then the result of the TMVA translation is evaluated, and then the probabilities from the sklean classifier are shown. A direct match isn't expected, but the trents (e.g. signal has the class2 column always larger than the bib or mj columns).

### What is wrong with this translation?
I'm using a slightly modified version of mlglue to do the translation. Yet, as you can see below, the MJ, Sig, and background come out almost alike. And the classifier works rather well when it is run internally. So... what is going on?

### BiB (class 0)
First lets look at BiB. We'll get some absolute numbers out of the weight calculations

In [18]:
events_to_use[0][:5][['RunNumber', 'EventNumber', 'JetPt', 'JetEta', 'JetPhi']]

,RunNumber,EventNumber,JetPt,JetEta,JetPhi
0,311170,927379924,191.9800,0.751230,1.51744
1,311170,933624178,129.4040,0.155269,-2.99099
2,311170,932123345,115.3560,-0.260263,3.09345
3,311170,929187568,155.0430,-0.640188,1.34265
4,311170,929187568,91.3555,0.247285,-2.10950


In [19]:
subsample = all_events[all_events_class.Class==0][:5]
subsample

,EnergyDensity,BIBDeltaTimingM,JetPt,HadronicLayer1Fraction,ShowerCenter,JetLat,FirstClusterRadius,JetLong,MaxTrackPt,PredictedLxy,BIBDeltaTimingP,PredictedLz,SumPtOfAllTracks
0,0.001389,15.063000,191.9800,0.745036,1784.0,0.859375,3376,0.953125,0.0000,2052.750,0.503155,1810.120,0.0000
1,0.002548,0.652127,129.4040,0.991674,952.0,0.283203,2496,0.312500,0.0000,1897.490,-2.198750,447.772,0.0000
2,0.002991,-3.603590,115.3560,1.000830,948.0,0.886719,2528,0.355469,0.0000,1961.130,0.726138,686.700,0.0000
3,0.001686,4.304950,155.0430,0.610702,916.0,0.964844,2656,0.660156,53.6083,719.184,13.267500,455.200,102.2630
4,0.005402,7.480070,91.3555,0.306610,218.0,0.867188,1840,0.695312,14.9536,741.596,2.930280,156.224,28.8335


In [20]:
bdtGeneral.eval(subsample)

array([[ 0.34370283,  0.32543338,  0.33086379],
       [ 0.35621121,  0.32911476,  0.31467402],
       [ 0.35374018,  0.32925487,  0.31700496],
       [ 0.3398921 ,  0.33985466,  0.32025324],
       [ 0.34024986,  0.339238  ,  0.32051214]])

In [21]:
bdt1.predict_proba(subsample)

array([[  9.76997972e-01,   4.15236339e-03,   1.88496644e-02],
       [  9.99629712e-01,   3.66707391e-04,   3.58069049e-06],
       [  9.99217720e-01,   7.67262076e-04,   1.50183803e-05],
       [  5.01894739e-01,   4.96971149e-01,   1.13411202e-03],
       [  5.72904826e-01,   4.25831800e-01,   1.26337376e-03]])

### MultiJet (class 1)

Ok, now lets look at MJ, which is class 1. Note that the run number is not pulled correctly here. :(

In [22]:
events_to_use[1][:5][['RunNumber', 'EventNumber', 'JetPt', 'JetEta', 'JetPhi']]

,RunNumber,EventNumber,JetPt,JetEta,JetPhi
0,284500,4157,46.9367,0.588283,-0.197074
1,284500,4160,58.6659,0.915437,0.622971
2,284500,4011,50.7740,0.682670,-0.738367
3,284500,4144,143.7050,0.985570,-1.977800
4,284500,4144,53.9551,-1.521310,-2.185010


In [23]:
subsample = all_events[all_events_class.Class==1][:5]
subsample

,EnergyDensity,BIBDeltaTimingM,JetPt,HadronicLayer1Fraction,ShowerCenter,JetLat,FirstClusterRadius,JetLong,MaxTrackPt,PredictedLxy,BIBDeltaTimingP,PredictedLz,SumPtOfAllTracks
532865,0.000222,15.43420,46.9367,0.903034,952.0,0.722656,2704,0.839844,9.27670,637.783,5.24863,383.406,19.73200
532866,0.003311,14.31030,58.6659,0.570251,362.0,0.933594,2512,0.988281,2.38707,675.068,2.18954,577.785,3.81886
532867,0.004364,14.50810,143.7050,0.533774,251.0,0.886719,2512,0.917969,32.18010,655.481,1.97326,741.678,85.33030
532868,0.011231,1.26047,53.9551,0.213439,50.0,0.648438,3312,0.320312,3.21193,458.212,21.01740,938.516,3.21193
532869,0.006104,14.42450,49.4316,0.814982,246.0,0.519531,2432,0.718750,7.72666,698.374,2.70632,596.755,7.72666


In [24]:
bdtGeneral.eval(subsample)

array([[ 0.33537332,  0.33734553,  0.32728115],
       [ 0.33458701,  0.33638723,  0.32902576],
       [ 0.33786141,  0.34215383,  0.31998476],
       [ 0.33438669,  0.33779387,  0.32781944],
       [ 0.33486572,  0.33826128,  0.326873  ]])

In [25]:
bdt1.predict_proba(subsample)

array([[ 0.34782621,  0.62590701,  0.02626678],
       [ 0.34786544,  0.59560304,  0.05653152],
       [ 0.22016778,  0.77899927,  0.00083295],
       [ 0.25781938,  0.71137117,  0.03080945],
       [ 0.26155353,  0.71816066,  0.02028582]])

### Signal (class 2)

Finally, lets look at signal

In [26]:
events_to_use[2][:5][['RunNumber', 'EventNumber', 'JetPt', 'JetEta', 'JetPhi']]

,RunNumber,EventNumber,JetPt,JetEta,JetPhi
0,284500,11890,406.728,0.654438,-1.088650
2,284500,11752,208.314,0.965120,1.787220
3,284500,11681,160.882,-1.915340,-2.662330
4,284500,11197,218.932,0.832934,-0.486832
5,284500,11197,168.861,-1.907410,2.612810


In [27]:
subsample = all_events[all_events_class.Class==2][:5]
subsample

,EnergyDensity,BIBDeltaTimingM,JetPt,HadronicLayer1Fraction,ShowerCenter,JetLat,FirstClusterRadius,JetLong,MaxTrackPt,PredictedLxy,BIBDeltaTimingP,PredictedLz,SumPtOfAllTracks
1066080,0.002594,21.44660,406.728,0.831321,1232.0,0.652344,3040,0.820312,0.0,2016.42,10.23040,1494.25,0.0
1066081,0.000660,36.55420,208.314,-0.001131,3936.0,0.371094,5440,0.322266,0.0,3293.99,8.81774,3404.06,0.0
1066082,0.005432,6.80703,160.882,0.151806,1064.0,0.777344,4992,0.820312,0.0,1043.40,38.70270,4139.80,0.0
1066083,0.001572,24.66940,218.932,0.724941,1296.0,0.488281,3296,0.832031,0.0,2106.88,10.13050,1962.46,0.0
1066084,0.009766,9.41108,168.861,0.043590,1120.0,0.371094,5088,0.025024,0.0,1269.28,41.90520,4506.63,0.0


In [28]:
bdtGeneral.eval(subsample)

array([[ 0.32960878,  0.32169185,  0.34869937],
       [ 0.32988545,  0.32174682,  0.34836773],
       [ 0.32663314,  0.32432261,  0.34904425],
       [ 0.33296141,  0.32428618,  0.3427524 ],
       [ 0.32955363,  0.32530829,  0.34513807]])

In [29]:
bdt1.predict_proba(subsample)

array([[  4.11166171e-03,   3.61948942e-04,   9.95526389e-01],
       [  4.91373374e-03,   4.04598928e-04,   9.94681667e-01],
       [  1.50730425e-03,   7.41994999e-04,   9.97750701e-01],
       [  5.94253339e-02,   4.24548868e-03,   9.36329177e-01],
       [  1.11763336e-02,   3.05988004e-03,   9.85763786e-01]])

## Plots of TMVA Translation Performance

In [30]:
%%time
predicted_tmva_weights = bdtGeneral.eval(perf_events)

Wall time: 23min 31s


In [31]:
fig = plt.figure(figsize=(15,10))

ax = plt.subplot(221)
ax.hist(predicted_tmva_weights[perf_events_class.Class == 0], label=('BiB', "MJ", 'HSS'), bins=50, histtype = 'step', range=[0.3, 0.37])
ax.set_xlabel('Weight')
ax.set_title('Class: BIB')
ax.legend()

ax = plt.subplot(222)
ax.hist(predicted_tmva_weights[perf_events_class.Class == 1], label=('BiB', "MJ", 'HSS'), bins=50, histtype = 'step', range=[0.3, 0.37])
ax.set_xlabel('Weight')
ax.set_title('Class: MJ')
ax.legend()

ax = plt.subplot(223)
ax.hist(predicted_tmva_weights[perf_events_class.Class == 2], label=('BiB', "MJ", 'HSS'), bins=50, histtype = 'step', range=[0.3, 0.37])
ax.set_xlabel('Weight')
ax.set_title('Class: HSS')
ax.legend()

Note in this version how everything is on top of itself. :(